In [1]:
from pymongo import MongoClient
import pandas as pd
import functions as f
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import json

# Data Cleaning

- Importamos base de datos de compañías

In [2]:
client = MongoClient("mongodb://localhost/companies")

In [3]:
db = client.get_database()

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

- Filtramos por empresas vivas y convertimos en DataFrame para poder operar

In [5]:
query = {'deadpooled_year':{'$eq':None}}

In [6]:
data = list(db["companies"].find(query,{"_id":0,"name":1,"category_code":1,"number_of_employees":1,"founded_year":1,"deadpooled_year":1,"total_money_raised":1,"offices":1}))

df = pd.DataFrame(data)

- Aplicamos la función que expande la columna de 'offices' y genera una nueva columna 'location' con el formato correcto de las coordenadas

In [7]:
df2 = f.officesClean(df)

- Exportamos como json para hacer consultas/modificar información

In [10]:
df2.to_json("../OUTPUT/first_order", orient="records")

- Importamos de nuevo el primer json limpio y filtramos solo por localizaciones con coordenadas

In [8]:
query2 = {'location':{'$ne':None}}

In [9]:
data2 = list(db["first_order"].find(query2,{"_id":0}))

first_order = pd.DataFrame(data2)

- Categorizamos las compañías según actividad para quedarnos solo con las tecnológicas

In [10]:
tech_comp = ["web","software","mobile","games_video","ecommerce","network_hosting","hardware","biotech","cleantech","analytics","semiconductor","photo_video","software","nanotech"]

first_order["tech_company"] = df["category_code"].apply(lambda x: "yes" if x in tech_comp else "no")

- Comprobamos que la mayoría de las empresas tecnológicas se sitúan en San Francisco, por tanto, ésta será la ciudad en la que buscaremos donde posicionarnos

In [11]:
tech = first_order['tech_company'] == 'yes'
filt = first_order[tech]

filt.city.value_counts().head(5)

San Francisco    399
New York         335
London           184
                 153
Austin            90
Name: city, dtype: int64

- Exportamos de nuevo el fichero ya limpio con todas las coordenadas y solo empresas con actividad en el área tecnológica

In [13]:
first_order.to_json("../OUTPUT/cleaned_companies", orient="records")

# Posicionamiento requisitos

- Importamos de nuevo la colección

In [12]:
query3 = {"$and":[{'city':'San Francisco','tech_company':'yes'}]}

In [15]:
data3 = list(db["cleaned_companies"].find(query3,{"_id":0}))

tech_companies = pd.DataFrame(data3)

In [18]:
tech_companies.to_json("../OUTPUT/tech_companies", orient="records")

- Creamos mapa con las coordenadas de San Francisco

In [19]:
start_lat = 37.773972
start_lon = -122.431297
heat_m = folium.Map(location=[start_lat, start_lon],tiles='cartodbpositron', zoom_start=15)

- Incluimos un mapa de calor con todas las compañías tecnológicas de San Francisco

In [20]:
sanF_group = folium.FeatureGroup(name='Sf_companies')
HeatMap(data=tech_companies[['latitude', 'longitude']]).add_to(sanF_group)
sanF_group.add_to(heat_m)

heat_m

- Obtenemos las coordenadas del aeropuerto de San Francisco

In [21]:
airport = f.googleApi('San Francisco Airport')

- Marcamos el punto en el mapa

In [22]:
uni_lon = airport['results'][0]['geometry']['location']['lng']
uni_lat = airport['results'][0]['geometry']['location']['lat']

Marker([uni_lat, uni_lon], icon=folium.Icon(color='blue', icon='plane', prefix='fa')).add_to(heat_m)
heat_m

- Importamos el csv que contiene todos los starbucks

In [24]:
starbucks = pd.read_csv('../INPUT/starbucks.csv')

- Filtramos por los starbucks que hay en San Francisco

In [25]:
filt = starbucks['City'] == 'San Francisco'
starbucks = starbucks[filt]

- Adaptamos las coordenadas

In [26]:
starbucks["location"] = starbucks[["Latitude","Longitude"]].apply(lambda x:f.coorFormat(x.Latitude,x.Longitude), axis=1)

- Guardamos como json para incluir colección en MongoDB

In [15]:
starbucks.to_json("../OUTPUT/starbucks", orient="records")

- Marcamos en el mapa todos los starbucks de San Francisco con su color representativo, el verde

In [27]:
for i in range(len(starbucks.Latitude)):
    Circle(
        location=[starbucks.iloc[i]['Latitude'],starbucks.iloc[i]['Longitude']],
        radius=15,
        color='green',
        fill=True,
        fill_color='green').add_to(heat_m)

In [28]:
heat_m

- Obtenemos las coordenadas de los pubs de San Francisco (Google solo me proporciona los primeros 20)

In [29]:
pubs = f.googlePlaces('pubs in san francisco')

- Guardamos como json para incluir colección en MongoDB

In [ ]:
with open('pubs.json', 'w') as fp:
    json.dump(pubs, fp)

- Sacamos las coordenadas de los pubs

In [30]:
pubs2 = []

for i in range(len(pubs['results'])):
    pubs2.append(pubs['results'][i]['geometry']['location'])

- Convertimos en DataFrame para adaptar las coordenadas

In [31]:
df_pubs = pd.DataFrame(pubs2)

In [32]:
df_pubs["location"] = df_pubs[["lat","lng"]].apply(lambda x:f.coorFormat(x.lat,x.lng), axis=1)

- Guardamos como csv como copia de seguridad

In [135]:
df_pubs.to_csv("../OUTPUT/pubs.csv")

- Marcamos en el mapa los pubs obtenidos, en color rojo

In [33]:
for i in range(len(df_pubs.lat)):
    Circle(
        location=[df_pubs.iloc[i]['lat'],df_pubs.iloc[i]['lng']],
        radius=15,
        color='red',
        fill=True,
        fill_color='red').add_to(heat_m)

In [34]:
heat_m

-  Obtenemos las coordenadas de los colegios en San Francisco (Google solo me proporciona los primeros 20)

In [35]:
schools = f.googlePlaces('schools in san francisco')

- Guardamos como json para incluir colección en MongoDB

In [142]:
with open('schools.json', 'w') as fp:
    json.dump(pubs, fp)

- Sacamos las coordenadas

In [36]:
schools2 = []

for i in range(len(schools['results'])):
    schools2.append(schools['results'][i]['geometry']['location'])

- Convertimos en DataFrame para adaptar las coordenadas

In [37]:
df_schools = pd.DataFrame(schools2)

In [38]:
df_schools["location"] = df_schools[["lat","lng"]].apply(lambda x:f.coorFormat(x.lat,x.lng), axis=1)

- Guardamos como csv como copia de seguridad

In [148]:
df_schools.to_csv("../OUTPUT/schools.csv")

- Marcamos en el mapa los colegios obtenidos, en color amarillo

In [39]:
for i in range(len(df_schools.lat)):
    Circle(
        location=[df_schools.iloc[i]['lat'],df_schools.iloc[i]['lng']],
        radius=15,
        color='yellow',
        fill=True,
        fill_color='yellow').add_to(heat_m)

In [40]:
heat_m

# Elección ubicación

In [42]:
tech_comp = tech_companies[['name','category_code','number_of_employees','founded_year','latitude','longitude','location']]

- Creamos una lista con las localizaciones de las compañías para poder compararlas con los demás parámetros

In [45]:
list_techcomp = []

for i in range(len(tech_comp)):
    list_techcomp.append(tech_comp['location'][i])

- Función genérica para las geoquerys

In [61]:
def queryLocation(parameter,collection,maxDistance=800,minDistance=0,field="location"):
    return list(db[collection].find({
       field: {
         "$near": {
           "$geometry": parameter,
           "$maxDistance": maxDistance,
           "$minDistance": minDistance
         }
       }
    }
    )
    )

- Función genérica para obtener el resultado de las geoquerys e incluirlo en el DataFrame

In [65]:
def geoquery(parameter,collection):
    matches = []
    for i in range(len(parameter)):
        check = queryLocation(parameter[i],collection)
        matches.append(len(check))
    return matches

- Generamos las geoquerys para cada compañía tecnológica de San Francisco en cada parámetros para obtener cuántas de éstos se sitúan a 800m como máximo

In [66]:
starbucks_matches = geoquery(list_techcomp,'starbucks')
pubs_matches = geoquery(list_techcomp,'pubs')
schools_matches = geoquery(list_techcomp,'schools')

- Incluimos los matches en el DataFrame de las compañías

In [71]:
tech_comp['starbucks800m'] = starbucks_matches
tech_comp['pubs800m'] = pubs_matches
tech_comp['schools800m'] = schools_matches

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [72]:
tech_comp.head()

,name,category_code,number_of_employees,founded_year,latitude,longitude,location,starbucks800m,pubs800m,schools800m
0,StumbleUpon,web,NaN,2002.0,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,...",2,0,1
1,Scribd,news,50.0,2007.0,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,...",27,4,0
2,Technorati,advertising,35.0,2002.0,37.779558,-122.393041,"{'type': 'Point', 'coordinates': [-122.393041,...",6,0,0
3,Kyte,games_video,40.0,2006.0,37.788482,-122.409173,"{'type': 'Point', 'coordinates': [-122.409173,...",8,4,0
4,Prosper,finance,NaN,2006.0,37.789760,-122.402524,"{'type': 'Point', 'coordinates': [-122.402524,...",27,4,0


- Creamos una columna con el sumatorio de las coincidencias (considero todas igual de importantes)

In [73]:
tech_comp['matches'] = tech_comp.starbucks800m + tech_comp.pubs800m + tech_comp.schools800m

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


- Ordenamos de mayor a menor

In [76]:
tech_comp.sort_values(by=['matches'], ascending = False).head()

,name,category_code,number_of_employees,founded_year,latitude,longitude,location,starbucks800m,pubs800m,schools800m,matches
52,OpenTable,web,550.0,1998.0,37.785647,-122.405265,"{'type': 'Point', 'coordinates': [-122.405265,...",34,5,0,39
244,Twonq,web,3.0,2008.0,37.794124,-122.403223,"{'type': 'Point', 'coordinates': [-122.403223,...",31,2,0,33
282,PicApp,web,NaN,2008.0,37.794020,-122.403124,"{'type': 'Point', 'coordinates': [-122.4031242...",31,2,0,33
265,Twonq,web,3.0,2008.0,37.794124,-122.403223,"{'type': 'Point', 'coordinates': [-122.403223,...",31,2,0,33
120,Vector Capital,None,NaN,1997.0,37.793473,-122.402667,"{'type': 'Point', 'coordinates': [-122.402667,...",31,2,0,33


- Con esto, obtenemos la empresa OpenTable con la mejor ubicación que satisface nuestras preferencias

In [89]:
company = tech_comp.name == 'OpenTable'
select = tech_comp[company]
final_decisionLat = select['latitude']
final_decisionLon = select['longitude']

In [90]:
print(final_decisionLat)
print(final_decisionLon)

52    37.785647
Name: latitude, dtype: float64
52   -122.405265
Name: longitude, dtype: float64


In [91]:
Marker([final_decisionLat, final_decisionLon], icon=folium.Icon(color='black', icon='briefcase', prefix='fa')).add_to(heat_m)

In [92]:
heat_m

In [93]:
heat_m.save('../OUTPUT/SanFrancisco_office')